In [4]:
import polars as pl
from methyl_jepa.infer import infer

ModuleNotFoundError: No module named 'methyl_jepa'

In [29]:
q_x = (
    pl.scan_parquet('../results/martin_inference.parquet')
    .head(1000)
    .with_columns((pl.col('read_name')+pl.col('pos').cast(pl.String)).alias('site_id'))
    .select(['site_id','strand', 'prob'])
    # .unpivot(index = 'siteid')
    )
df_x = q_x.collect()
df_x 


site_id,strand,prob
str,str,f32
"""m84108_250708_203104_s1/908995…","""fwd""",0.402191
"""m84108_250708_203104_s1/908995…","""rev""",0.801893
"""m84108_250708_203104_s1/908995…","""fwd""",0.679787
"""m84108_250708_203104_s1/908995…","""rev""",0.770652
"""m84108_250708_203104_s1/908995…","""fwd""",0.529284
…,…,…
"""m84108_250708_203104_s1/190453…","""rev""",0.837016
"""m84108_250708_203104_s1/190453…","""fwd""",0.979658
"""m84108_250708_203104_s1/190453…","""rev""",0.976568


In [31]:
q_join = (
    pl.scan_parquet('../data/01_processed/inference_sets/martin.parquet')
    .head(1000)
    .with_columns((pl.col('read_name')+pl.col('cg_pos').cast(pl.String)).alias('site_id'))
    .join(other=q_x, on='site_id', how='inner')
    )
df_join = q_join.collect()
df_join

read_name,cg_pos,seq,qual,np,sm,sx,fi,fp,ri,rp,site_id,strand,prob
str,u16,str,list[u8],u8,list[u8],list[u8],list[u16],list[u16],list[u16],list[u16],str,str,f32
"""m84108_250708_203104_s1/908995…",41,"""AGAAAAAACAAGTAACGTGCTTCCTCGACA…","[93, 93, … 93]",20,"[18, 18, … 20]","[2, 2, … 0]","[11, 8, … 22]","[14, 19, … 19]","[14, 25, … 14]","[18, 17, … 17]","""m84108_250708_203104_s1/908995…","""fwd""",0.402191
"""m84108_250708_203104_s1/908995…",41,"""AGAAAAAACAAGTAACGTGCTTCCTCGACA…","[93, 93, … 93]",20,"[18, 18, … 20]","[2, 2, … 0]","[11, 8, … 22]","[14, 19, … 19]","[14, 25, … 14]","[18, 17, … 17]","""m84108_250708_203104_s1/908995…","""rev""",0.801893
"""m84108_250708_203104_s1/908995…",51,"""AGTAACGTGCTTCCTCGACAAACAAAGCTG…","[93, 93, … 67]",20,"[17, 19, … 16]","[3, 0, … 0]","[17, 13, … 14]","[8, 17, … 18]","[16, 13, … 19]","[20, 11, … 12]","""m84108_250708_203104_s1/908995…","""fwd""",0.679787
"""m84108_250708_203104_s1/908995…",51,"""AGTAACGTGCTTCCTCGACAAACAAAGCTG…","[93, 93, … 67]",20,"[17, 19, … 16]","[3, 0, … 0]","[17, 13, … 14]","[8, 17, … 18]","[16, 13, … 19]","[20, 11, … 12]","""m84108_250708_203104_s1/908995…","""rev""",0.770652
"""m84108_250708_203104_s1/908995…",109,"""TAAAAAAATACTTAACGGAGTTTTTCAAGT…","[93, 43, … 93]",20,"[18, 14, … 17]","[1, 1, … 1]","[14, 22, … 14]","[19, 14, … 11]","[12, 9, … 3]","[19, 23, … 34]","""m84108_250708_203104_s1/908995…","""fwd""",0.529284
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""m84108_250708_203104_s1/190453…",1591,"""CCAGTGAGGACATGGCGAGTGACGAGGAGC…","[93, 93, … 75]",13,"[13, 14, … 14]","[0, 0, … 0]","[6, 18, … 31]","[12, 12, … 13]","[90, 19, … 20]","[16, 17, … 17]","""m84108_250708_203104_s1/190453…","""rev""",0.837016
"""m84108_250708_203104_s1/190453…",1598,"""GGACATGGCGAGTGACGAGGAGCACACGGT…","[93, 93, … 93]",13,"[11, 14, … 14]","[1, 0, … 0]","[27, 24, … 24]","[11, 18, … 11]","[19, 31, … 16]","[8, 21, … 13]","""m84108_250708_203104_s1/190453…","""fwd""",0.979658
"""m84108_250708_203104_s1/190453…",1598,"""GGACATGGCGAGTGACGAGGAGCACACGGT…","[93, 93, … 93]",13,"[11, 14, … 14]","[1, 0, … 0]","[27, 24, … 24]","[11, 18, … 11]","[19, 31, … 16]","[8, 21, … 13]","""m84108_250708_203104_s1/190453…","""rev""",0.976568
